In [1]:
import pandas as pd

df = pd.concat((
    pd.read_csv('./data/October_missions_full.csv'),
), ignore_index=True)

df['mission'] = df['type'] + '_' + df['target'].astype(str)

df = df[['user', 'mission', 'createdAtT', 'type', 'target', 'performance']]
df['createdAtT'] = pd.to_datetime(df['createdAtT'], unit='ms').dt.date
df = df.groupby('user').filter(lambda x: len(x['createdAtT'].unique()) > 2)

df['user'] = df['user'].astype('category').cat.codes
df['mission'] = df['mission'].astype('category')
df['missionID'] = df['mission'].cat.codes
df['type'] = df['type'].astype('category')
df['performance'] = df['performance'].apply(lambda x: 0.7 <= x <= 1.05).astype(int)

df.sort_values(by=['createdAtT', 'user'], inplace=True, ignore_index=True)
df

,user,mission,createdAtT,type,target,performance,missionID
0,0,action_1,2024-10-01,action,1,1,0
1,0,quiz_5,2024-10-01,quiz,5,1,28
2,0,exp_100,2024-10-01,exp,100,0,18
3,2,episode_1,2024-10-01,episode,1,1,12
4,2,quiz_1,2024-10-01,quiz,1,1,24
...,...,...,...,...,...,...,...
70899,1822,episode_5,2024-10-31,episode,5,0,16
70900,1822,activity_5,2024-10-31,activity,5,0,7
70901,1823,episode_5,2024-10-31,episode,5,0,16
70902,1823,mobility_1,2024-10-31,mobility,1,0,20


In [ ]:
from src.tree.tree import TreeNode

missions = df[['missionID', 'type', 'target']].drop_duplicates()
root = TreeNode('root')
for name, group in missions.groupby('type', observed=True):
    node = TreeNode(name)
    root.add_child(node)
    for _, mission in group.iterrows():
        node.add_child(TreeNode(mission.to_dict()))

print(root)

ModuleNotFoundError: No module named 'tree'

In [ ]:
import torch
import numpy as np
import torch.nn as nn
from scipy.special import softmax
from bandit.policy import Policy
from models import mf